# Imports

In [1]:
import csv 
import sys
import zipcodes
from datetime import date

print(f'Version Info: {sys.version_info}')

Version Info: sys.version_info(major=3, minor=8, micro=6, releaselevel='final', serial=0)


# Opening Dataset

### Storing in a dictionary 

Write function doesn't seem to work if file is open. 

In [2]:
# function to read file from csv 
def load_data(csv_file="DataSet.csv"):
    data = [] 
    with open(csv_file, 'r') as file: 
        for line in csv.DictReader(file): 
            data.append(line) 
    return data

# function to write dict to file
def write_data(data, csv_file="DataSet.csv"):
    csv_columns = list(data[0].keys())
    try:
        with open(csv_file, 'w') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
            writer.writeheader()
            for line in data:
                writer.writerow(line)
    except IOError:
        print("I/O error")

data = load_data()

### Data cols

All the data categories found after parsing

In [3]:
for col in data[0].keys():
    print(f"\"{col}\"")

"PersonID"
"Date"
"Time"
"HHState"
"Zipcode"
"Sex"
"Age"
"HHsize"
"Kids"
"Auto_own"
"Married"
"Educ"
"HType"
"Worker"
"commute_before"
"commute_after"
"Transmode_before"
"changemode"
"Transmode_after"
"Commute_min"
"PhyAc_Before"
"Exe_Before"
"PhyAc_After"
"Exe_After"
"Groc_Shp"
"Outside_Ac"
"Outside_Purp"
"ONL_SHP"
"ONL_SHP_AFTR"


# Sanitization

In [41]:
for row in data:
    row['HHState'] = row['HHState'].replace(" ", "").upper()
    row['Outside_Purp'] = row['Outside_Purp'].strip()
    try:
        int(row['Commute_min'])
        if("n/a" == row['Commute_min'].lower()):
            row['Commute_min'] = -8
    except AttributeError:
        pass

### Verification functions

In [33]:
def verify(data, valid_values, *fields):
    error_found = False
    for row in data:
        for field in fields:
            try:
                if(row[field] not in valid_values):
                    raise ValueError(f'{field}:{row[field]} Invalid field value')
            except ValueError as e:
                error_found = True
                print(f'[{row["PersonID"]}]', e)
    return not error_found
                
def verify_int(data, *fields):
    error_found = False
    for row in data:
        for field in fields:
            try:
                if(int(row[field]) != -8 and int(row[field]) < 0):
                    raise TypeError(f'{field}:{row[field]} Invalid field value')
            except TypeError as e:
                error_found = True
                print(f'[{row["PersonID"]}]', e)
            except ValueError as e:
                error_found = True
                print(f'[{row["PersonID"]}] {field}:{row[field]} Invalid field value')
    return not error_found


### User Id Ordering and Verification

Check for no duplicate user id's and reorders the list of dicts from smallest id to largest

In [6]:
#checks for duplicates and build set of values
ids = set()
reordered_data = [""] * (len(data)) 
for row in data:
    try:
        if(row['PersonID'] in ids):
            raise ValueError("repeated id")
        else:
            ids.add(int(row['PersonID']))
            reordered_data[int(row['PersonID'])-1] = row
    except ValueError as e:
        print(f'[{row["PersonID"]}]', e)

data = reordered_data
print(f'Number of id\'s [{len(ids)}] vs number of rows: [{len(data)}]')

Number of id's [564] vs number of rows: [564]


### Time and date

Does this need verification? Honestly not really useful for anything...

### Zipcode/State

In [7]:
# for every line in the dict
for row in data:
    try:
        #grab that rows zipcode
        zip=str(row['Zipcode'])
        
        # check the zipcode field isn't empty
        if(zip == ""):
            raise ValueError('Zipcode field is empty')
            
        # check the zipcode is real
        if(not zipcodes.is_real(zip)):
            raise ValueError(f'Zipcode {zip} is not real')
            
        # check the zipcode matches the state
        if(zipcodes.matching(zip)[0]['state'] != row['HHState']):
            raise ValueError(f'For zipcode {zip}, User defined state: {row["HHState"]} does not equal Zipcode defined state:{zipcodes.matching(zip)[0]["state"]}')
            
    except Exception as e:
        print(f'[{row["PersonID"]}]', e)

[31] Zipcode 00000 is not real
[82] Zipcode 2016 is not real
[161] Zipcode field is empty
[233] Zipcode field is empty
[272] Invalid format, zipcode must be of the format: "#####" or "#####-####"


### Fields greater then zero

In [42]:
verify_int(data, "Age", "HHsize", "Kids", "Auto_own", "Commute_min")

[6] Age: Invalid field value
[6] HHsize: Invalid field value
[6] Auto_own: Invalid field value
[483] Kids: Invalid field value
[500] Age:01-21-1930 Invalid field value
[505] Age:30+ Invalid field value


False

### Fields only equal to 1,2,3,4

In [9]:
valid_values = set(["1", "2", "3", "4"])
if(verify(data, valid_values, "Worker", "PhyAc_Before", "Exe_Before", "PhyAc_After", "Outside_Ac")):
    print("No Errors found")

[220] PhyAc_Before:N/A Invalid field value


### Fields only equal to 1,2,3,4,5

In [10]:
valid_values = set(["1", "2", "3", "4", "5"])
if(verify(data, valid_values, "HType", "Groc_Shp")):
    print("No Errors found")

No Errors found


### Fields only equal to 1,2,3,-8, 

In [11]:
valid_values = set(["1","2","3","-8"])
if(verify(data, valid_values, "Married", "commute_before", "ONL_SHP_AFTR")):
    print("No Errors found")

No Errors found


### Fields only equal to 1,2,3,4,-8 

In [12]:
valid_values = set(["1","2","3","4","-8"])
if(verify(data, valid_values, "Educ")):
    print("No Errors found")

No Errors found


### Fields only equal to 1,2,-8,-9 

In [13]:
valid_values = set(["1","2","-8","-9"])
if(verify(data, valid_values, "Sex")):
    print("No Errors found")

No Errors found


### Fields only equal to 1,2,3,4,5,6,7,-8

In [14]:
valid_values = set(["1","2","3","4","5","6","7","-8"])
if(verify(data, valid_values, "commute_after", "Transmode_before", "Transmode_after")):
    print("No Errors found")

No Errors found


### Fields only equal to 1,2,-8

In [15]:
valid_values = set(["1","2","-8"])
if(verify(data, valid_values, "changemode", "ONL_SHP")):
    print("No Errors found")

No Errors found


### Fields only equal to 1,2,3,4,5,6,7

In [16]:
valid_values = set(["1","2","3","4","5","6","7"])
if(verify(data, valid_values, "Exe_After")):
    print("No Errors found")

No Errors found


# Update Data set on disk

write_data(data)